Did you know that the average return from investing in stocks is 10% per year! But who wants to be average?! 

You have been asked to support an investment firm by analyzing trends in high-growth companies. They are interested in understanding which industries are producing the highest valuations and the rate at which new high-value companies are emerging. Providing them with this information gives them a competitive insight as to industry trends and how they should structure their portfolio looking forward.

You have been given access to their `unicorns` database, which contains the following tables:

`dates`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| date_joined  | The date that the company became a unicorn.  |
| year_founded | The year that the company was founded.       |

`funding`
| Column           | Description                                  |
|----------------- |--------------------------------------------- |
| company_id       | A unique ID for the company.                 |
| valuation        | Company value in US dollars.                 |
| funding          | The amount of funding raised in US dollars.  |
| select_investors | A list of key investors in the company.      |

`industries`
| Column       | Description                                  |
|------------- |--------------------------------------------- |
| company_id   | A unique ID for the company.                 |
| industry     | The industry that the company operates in.   |

`companies`
| Column       | Description                                       |
|------------- |-------------------------------------------------- |
| company_id   | A unique ID for the company.                      |
| company      | The name of the company.                          |
| city         | The city where the company is headquartered.      |
| country      | The country where the company is headquartered.   |
| continent    | The continent where the company is headquartered. |


In [6]:
-- This query will become our CTE for the main query. Here I'm checking that it works correctly.
-- Select the industry and count the number of unicorns in each industry
SELECT i.industry, COUNT(*) AS num_unicorns
FROM companies AS c
-- Join the companies, industries, and dates tables on the company_id column
LEFT JOIN industries AS i ON c.company_id = i.company_id
LEFT JOIN dates AS d ON c.company_id = d.company_id
-- Filter the rows based on the date_joined column to include only years 2019, 2020, and 2021
WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
-- Group the results by the industry column
GROUP BY i.industry
-- Order the results by the num_unicorns column in descending order
ORDER BY num_unicorns DESC
-- Limit the results to the top 3 industries
LIMIT 3

,industry,num_unicorns
0,Fintech,173
1,Internet software & services,152
2,E-commerce & direct-to-consumer,75


In [10]:
-- Create CTE to identify the top three industries in question
WITH top_industries AS (
  -- Select the industry and count the number of unicorns in each industry
  SELECT i.industry, COUNT(*) AS num_unicorns
  FROM companies AS c
  -- Join the companies, industries, and dates tables on the company_id column
  LEFT JOIN industries AS i ON c.company_id = i.company_id
  LEFT JOIN dates AS d ON c.company_id = d.company_id
  -- Filter the rows based on the date_joined column to include only years 2019, 2020, and 2021
  WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019, 2020, 2021)
  -- Group the results by the industry column
  GROUP BY i.industry
  -- Order the results by the num_unicorns column in descending order
  ORDER BY num_unicorns DESC
  -- Limit the results to the top 3 industries
  LIMIT 3
)
-- Select industry, year (extracted from date_joined), count of new unicorns, and avg valuation (converted to billions and rounded)
SELECT 
  i.industry,
  EXTRACT(YEAR FROM d.date_joined) AS year, 
  COUNT(*) AS num_unicorns, 
  ROUND(AVG(f.valuation::numeric/1000000000),2) as average_valuation_billions
-- Join all tables on company_id
FROM
  companies AS c
  LEFT JOIN dates AS d ON c.company_id = d.company_id
  LEFT JOIN industries AS i ON c.company_id = i.company_id
  LEFT JOIN funding AS f ON c.company_id = f.company_id
-- Filter for target years, and the top performing industries (from CTE)
WHERE EXTRACT(YEAR FROM d.date_joined) IN (2019,2020,2021) AND i.industry IN (SELECT industry FROM top_industries)
GROUP BY i.industry, year
ORDER BY i.industry, year DESC;

,industry,year,num_unicorns,average_valuation_billions
0,E-commerce & direct-to-consumer,2021,47,2.47
1,E-commerce & direct-to-consumer,2020,16,4.00
2,E-commerce & direct-to-consumer,2019,12,2.58
3,Fintech,2021,138,2.75
4,Fintech,2020,15,4.33
5,Fintech,2019,20,6.80
6,Internet software & services,2021,119,2.15
7,Internet software & services,2020,20,4.35
8,Internet software & services,2019,13,4.23
